## 1. Import Libraries & Load Data <a id='1'></a>

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score, roc_curve,
    mean_squared_error, mean_absolute_error, r2_score
)

# Classification Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Regression Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor

# Additional tools
import joblib

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully!")

In [ ]:
# Load the transformed dataset
df = pd.read_csv('Crime_Data_Transformed.csv')

# Convert date columns
df['Date Rptd'] = pd.to_datetime(df['Date Rptd'])
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])

print(f"✓ Dataset loaded successfully!")
print(f"  Shape: {df.shape}")
print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

df.head()

## 2. Feature Engineering <a id='2'></a>

Prepare features for machine learning models.

In [ ]:
# Create a copy for modeling
df_model = df.copy()

# Encode categorical variables
label_encoders = {}

categorical_cols = ['AREA NAME', 'day_name', 'month_name', 'time_period', 
                    'victim_age_group', 'location_type', 'weapon_category']

for col in categorical_cols:
    if col in df_model.columns:
        le = LabelEncoder()
        df_model[f'{col}_encoded'] = le.fit_transform(df_model[col].astype(str))
        label_encoders[col] = le

print("✓ Categorical variables encoded")

# Create additional features
df_model['is_serious_crime'] = (df_model['crime_severity'] == 'Part 1 - Serious Crime').astype(int)
df_model['is_violent'] = (df_model['crime_category'] == 'Violent Crime').astype(int)
df_model['is_property'] = (df_model['crime_category'] == 'Property Crime').astype(int)

# Time-based features
df_model['is_night'] = ((df_model['hour'] >= 0) & (df_model['hour'] < 6)).astype(int)
df_model['is_rush_hour'] = ((df_model['hour'] >= 7) & (df_model['hour'] <= 9) | 
                             (df_model['hour'] >= 17) & (df_model['hour'] <= 19)).astype(int)

print("✓ Additional features created")
print(f"\nTotal features: {df_model.shape[1]}")

In [ ]:
# Define feature sets for different models

# Numerical features
numerical_features = [
    'Vict Age', 'AREA', 'hour', 'month', 'quarter', 'year',
    'is_weekend', 'reporting_delay_days', 'area_risk_score',
    'population', 'median_income', 'area_size_sq_miles',
    'crimes_per_1000', 'is_night', 'is_rush_hour'
]

# Encoded categorical features
encoded_features = [f'{col}_encoded' for col in categorical_cols if col in df_model.columns]

# Combined features
all_features = numerical_features + encoded_features

# Filter features that exist in the dataframe
available_features = [f for f in all_features if f in df_model.columns]

print(f"Available features for modeling: {len(available_features)}")
print(f"\nFeature list:")
for i, feat in enumerate(available_features, 1):
    print(f"  {i}. {feat}")

## 3. Model 1: Crime Category Classification <a id='3'></a>

Predict the category of crime (Violent, Property, or Other).

In [ ]:
print("="*80)
print("MODEL 1: CRIME CATEGORY CLASSIFICATION")
print("="*80)

# Prepare data
X = df_model[available_features].fillna(0)
y = df_model['crime_category']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nClass distribution:")
print(y_train.value_counts())

In [ ]:
# Train multiple classifiers
classifiers = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10)
}

results_category = {}

print("\nTraining classifiers...\n")

for name, clf in classifiers.items():
    print(f"Training {name}...")
    
    # Train model
    clf.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred = clf.predict(X_test_scaled)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    results_category[name] = {
        'model': clf,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'predictions': y_pred
    }
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  F1-Score: {f1:.4f}\n")

print("✓ All classifiers trained successfully!")

In [ ]:
# Compare models
comparison_df = pd.DataFrame({
    'Model': list(results_category.keys()),
    'Accuracy': [r['accuracy'] for r in results_category.values()],
    'Precision': [r['precision'] for r in results_category.values()],
    'Recall': [r['recall'] for r in results_category.values()],
    'F1-Score': [r['f1_score'] for r in results_category.values()]
}).sort_values('F1-Score', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON - CRIME CATEGORY CLASSIFICATION")
print("="*80)
print(comparison_df.to_string(index=False))

# Best model
best_model_name = comparison_df.iloc[0]['Model']
best_model = results_category[best_model_name]['model']
print(f"\n🏆 Best Model: {best_model_name}")
print(f"   F1-Score: {results_category[best_model_name]['f1_score']:.4f}")

In [ ]:
# Visualize results for best model
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Confusion Matrix
cm = confusion_matrix(y_test, results_category[best_model_name]['predictions'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=best_model.classes_, yticklabels=best_model.classes_)
axes[0].set_title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Model Comparison
comparison_df.set_index('Model')[['Accuracy', 'Precision', 'Recall', 'F1-Score']].plot(
    kind='bar', ax=axes[1], width=0.8
)
axes[1].set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Score')
axes[1].set_xlabel('Model')
axes[1].legend(loc='lower right')
axes[1].set_ylim([0.5, 1.0])
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('model1_crime_category_classification.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization saved: model1_crime_category_classification.png")

In [ ]:
# Detailed classification report
print("\n" + "="*80)
print(f"DETAILED CLASSIFICATION REPORT - {best_model_name}")
print("="*80)
print(classification_report(y_test, results_category[best_model_name]['predictions']))

## 4. Model 2: Crime Severity Prediction <a id='4'></a>

Predict whether a crime is Part 1 (Serious) or Part 2 (Less Serious).

In [ ]:
print("="*80)
print("MODEL 2: CRIME SEVERITY PREDICTION")
print("="*80)

# Prepare data
X_severity = df_model[available_features].fillna(0)
y_severity = df_model['is_serious_crime']

# Split data
X_train_sev, X_test_sev, y_train_sev, y_test_sev = train_test_split(
    X_severity, y_severity, test_size=0.2, random_state=42, stratify=y_severity
)

# Scale features
scaler_sev = StandardScaler()
X_train_sev_scaled = scaler_sev.fit_transform(X_train_sev)
X_test_sev_scaled = scaler_sev.transform(X_test_sev)

print(f"\nTraining set: {X_train_sev.shape}")
print(f"Test set: {X_test_sev.shape}")
print(f"\nClass distribution:")
print(y_train_sev.value_counts())

In [ ]:
# Train classifiers for severity
classifiers_sev = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

results_severity = {}

print("\nTraining classifiers...\n")

for name, clf in classifiers_sev.items():
    print(f"Training {name}...")
    
    clf.fit(X_train_sev_scaled, y_train_sev)
    y_pred = clf.predict(X_test_sev_scaled)
    y_pred_proba = clf.predict_proba(X_test_sev_scaled)[:, 1]
    
    accuracy = accuracy_score(y_test_sev, y_pred)
    precision = precision_score(y_test_sev, y_pred)
    recall = recall_score(y_test_sev, y_pred)
    f1 = f1_score(y_test_sev, y_pred)
    auc = roc_auc_score(y_test_sev, y_pred_proba)
    
    results_severity[name] = {
        'model': clf,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  AUC-ROC: {auc:.4f}\n")

print("✓ All classifiers trained successfully!")

In [ ]:
# Compare models
comparison_sev = pd.DataFrame({
    'Model': list(results_severity.keys()),
    'Accuracy': [r['accuracy'] for r in results_severity.values()],
    'Precision': [r['precision'] for r in results_severity.values()],
    'Recall': [r['recall'] for r in results_severity.values()],
    'F1-Score': [r['f1_score'] for r in results_severity.values()],
    'AUC-ROC': [r['auc'] for r in results_severity.values()]
}).sort_values('AUC-ROC', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON - CRIME SEVERITY PREDICTION")
print("="*80)
print(comparison_sev.to_string(index=False))

best_model_sev = comparison_sev.iloc[0]['Model']
print(f"\n🏆 Best Model: {best_model_sev}")
print(f"   AUC-ROC: {results_severity[best_model_sev]['auc']:.4f}")

In [ ]:
# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curves
for name in results_severity.keys():
    fpr, tpr, _ = roc_curve(y_test_sev, results_severity[name]['probabilities'])
    auc = results_severity[name]['auc']
    axes[0].plot(fpr, tpr, label=f'{name} (AUC = {auc:.4f})', linewidth=2)

axes[0].plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curves - Severity Prediction', fontsize=14, fontweight='bold')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Confusion Matrix for best model
cm = confusion_matrix(y_test_sev, results_severity[best_model_sev]['predictions'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['Part 2', 'Part 1'], yticklabels=['Part 2', 'Part 1'])
axes[1].set_title(f'Confusion Matrix - {best_model_sev}', fontsize=14, fontweight='bold')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig('model2_crime_severity_prediction.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization saved: model2_crime_severity_prediction.png")

## 5. Model 3: Weapon Involvement Prediction <a id='5'></a>

Predict whether a weapon will be involved in a crime.

In [ ]:
print("="*80)
print("MODEL 3: WEAPON INVOLVEMENT PREDICTION")
print("="*80)

# Prepare data (exclude weapon-related features from predictors)
weapon_features = [f for f in available_features if 'weapon' not in f.lower()]
X_weapon = df_model[weapon_features].fillna(0)
y_weapon = df_model['weapon_involved']

# Split data
X_train_weap, X_test_weap, y_train_weap, y_test_weap = train_test_split(
    X_weapon, y_weapon, test_size=0.2, random_state=42, stratify=y_weapon
)

# Scale features
scaler_weap = StandardScaler()
X_train_weap_scaled = scaler_weap.fit_transform(X_train_weap)
X_test_weap_scaled = scaler_weap.transform(X_test_weap)

print(f"\nFeatures used: {len(weapon_features)}")
print(f"Training set: {X_train_weap.shape}")
print(f"Test set: {X_test_weap.shape}")
print(f"\nClass distribution:")
print(y_train_weap.value_counts())

In [ ]:
# Train models
classifiers_weapon = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

results_weapon = {}

print("\nTraining classifiers...\n")

for name, clf in classifiers_weapon.items():
    print(f"Training {name}...")
    
    clf.fit(X_train_weap_scaled, y_train_weap)
    y_pred = clf.predict(X_test_weap_scaled)
    y_pred_proba = clf.predict_proba(X_test_weap_scaled)[:, 1]
    
    accuracy = accuracy_score(y_test_weap, y_pred)
    precision = precision_score(y_test_weap, y_pred)
    recall = recall_score(y_test_weap, y_pred)
    f1 = f1_score(y_test_weap, y_pred)
    auc = roc_auc_score(y_test_weap, y_pred_proba)
    
    results_weapon[name] = {
        'model': clf,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'predictions': y_pred
    }
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  F1-Score: {f1:.4f}\n")

print("✓ All classifiers trained successfully!")

In [ ]:
# Compare models
comparison_weapon = pd.DataFrame({
    'Model': list(results_weapon.keys()),
    'Accuracy': [r['accuracy'] for r in results_weapon.values()],
    'Precision': [r['precision'] for r in results_weapon.values()],
    'Recall': [r['recall'] for r in results_weapon.values()],
    'F1-Score': [r['f1_score'] for r in results_weapon.values()],
    'AUC-ROC': [r['auc'] for r in results_weapon.values()]
}).sort_values('F1-Score', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON - WEAPON INVOLVEMENT PREDICTION")
print("="*80)
print(comparison_weapon.to_string(index=False))

best_model_weapon = comparison_weapon.iloc[0]['Model']
print(f"\n🏆 Best Model: {best_model_weapon}")
print(f"   F1-Score: {results_weapon[best_model_weapon]['f1_score']:.4f}")

## 6. Model 4: Crime Occurrence Prediction (Time Series) <a id='6'></a>

Predict the number of crimes in future time periods.

In [ ]:
print("="*80)
print("MODEL 4: CRIME OCCURRENCE PREDICTION (TIME SERIES)")
print("="*80)

# Create daily crime counts
df_ts = df.set_index('DATE OCC').sort_index()
daily_crimes = df_ts.resample('D').size().reset_index()
daily_crimes.columns = ['date', 'crime_count']

# Feature engineering for time series
daily_crimes['year'] = daily_crimes['date'].dt.year
daily_crimes['month'] = daily_crimes['date'].dt.month
daily_crimes['day'] = daily_crimes['date'].dt.day
daily_crimes['dayofweek'] = daily_crimes['date'].dt.dayofweek
daily_crimes['quarter'] = daily_crimes['date'].dt.quarter
daily_crimes['is_weekend'] = (daily_crimes['dayofweek'] >= 5).astype(int)

# Create lag features
for lag in [1, 7, 30]:
    daily_crimes[f'lag_{lag}'] = daily_crimes['crime_count'].shift(lag)

# Rolling averages
daily_crimes['rolling_7'] = daily_crimes['crime_count'].rolling(window=7).mean()
daily_crimes['rolling_30'] = daily_crimes['crime_count'].rolling(window=30).mean()

# Drop rows with NaN
daily_crimes = daily_crimes.dropna()

print(f"\nTime series data shape: {daily_crimes.shape}")
print(f"Date range: {daily_crimes['date'].min()} to {daily_crimes['date'].max()}")
daily_crimes.head(10)

In [ ]:
# Prepare features for regression
ts_features = ['year', 'month', 'day', 'dayofweek', 'quarter', 'is_weekend',
               'lag_1', 'lag_7', 'lag_30', 'rolling_7', 'rolling_30']

X_ts = daily_crimes[ts_features]
y_ts = daily_crimes['crime_count']

# Split data (time series split)
split_idx = int(len(X_ts) * 0.8)
X_train_ts = X_ts[:split_idx]
X_test_ts = X_ts[split_idx:]
y_train_ts = y_ts[:split_idx]
y_test_ts = y_ts[split_idx:]

print(f"Training set: {X_train_ts.shape}")
print(f"Test set: {X_test_ts.shape}")

In [ ]:
# Train regression models
regressors = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression()
}

results_ts = {}

print("\nTraining regression models...\n")

for name, reg in regressors.items():
    print(f"Training {name}...")
    
    reg.fit(X_train_ts, y_train_ts)
    y_pred = reg.predict(X_test_ts)
    
    mae = mean_absolute_error(y_test_ts, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_ts, y_pred))
    r2 = r2_score(y_test_ts, y_pred)
    
    results_ts[name] = {
        'model': reg,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'predictions': y_pred
    }
    
    print(f"  MAE: {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R²: {r2:.4f}\n")

print("✓ All regressors trained successfully!")

In [ ]:
# Compare models
comparison_ts = pd.DataFrame({
    'Model': list(results_ts.keys()),
    'MAE': [r['mae'] for r in results_ts.values()],
    'RMSE': [r['rmse'] for r in results_ts.values()],
    'R²': [r['r2'] for r in results_ts.values()]
}).sort_values('MAE', ascending=True)

print("\n" + "="*80)
print("MODEL COMPARISON - CRIME OCCURRENCE PREDICTION")
print("="*80)
print(comparison_ts.to_string(index=False))

best_model_ts = comparison_ts.iloc[0]['Model']
print(f"\n🏆 Best Model: {best_model_ts}")
print(f"   MAE: {results_ts[best_model_ts]['mae']:.4f}")
print(f"   R²: {results_ts[best_model_ts]['r2']:.4f}")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Plot actual vs predicted for best model
test_dates = daily_crimes.iloc[split_idx:]['date']
axes[0].plot(test_dates, y_test_ts.values, label='Actual', linewidth=2, alpha=0.7)
axes[0].plot(test_dates, results_ts[best_model_ts]['predictions'], 
             label='Predicted', linewidth=2, alpha=0.7)
axes[0].set_title(f'Crime Occurrence Prediction - {best_model_ts}', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date', fontsize=12)
axes[0].set_ylabel('Number of Crimes', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Residuals plot
residuals = y_test_ts.values - results_ts[best_model_ts]['predictions']
axes[1].scatter(results_ts[best_model_ts]['predictions'], residuals, alpha=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_title('Residuals Plot', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted Values', fontsize=12)
axes[1].set_ylabel('Residuals', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model4_crime_occurrence_prediction.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization saved: model4_crime_occurrence_prediction.png")

## 7. Model 5: Area Risk Score Regression <a id='7'></a>

Predict the risk score of an area based on various factors.

In [ ]:
print("="*80)
print("MODEL 5: AREA RISK SCORE PREDICTION")
print("="*80)

# Prepare data (exclude area_risk_score from predictors)
risk_features = [f for f in available_features if 'risk' not in f.lower()]
X_risk = df_model[risk_features].fillna(0)
y_risk = df_model['area_risk_score']

# Split data
X_train_risk, X_test_risk, y_train_risk, y_test_risk = train_test_split(
    X_risk, y_risk, test_size=0.2, random_state=42
)

print(f"\nTraining set: {X_train_risk.shape}")
print(f"Test set: {X_test_risk.shape}")
print(f"\nTarget statistics:")
print(y_train_risk.describe())

In [ ]:
# Train regression models
regressors_risk = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42)
}

results_risk = {}

print("\nTraining regression models...\n")

for name, reg in regressors_risk.items():
    print(f"Training {name}...")
    
    reg.fit(X_train_risk, y_train_risk)
    y_pred = reg.predict(X_test_risk)
    
    mae = mean_absolute_error(y_test_risk, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_risk, y_pred))
    r2 = r2_score(y_test_risk, y_pred)
    
    results_risk[name] = {
        'model': reg,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'predictions': y_pred
    }
    
    print(f"  MAE: {mae:.4f}")
    print(f"  R²: {r2:.4f}\n")

print("✓ All regressors trained successfully!")

In [ ]:
# Compare models
comparison_risk = pd.DataFrame({
    'Model': list(results_risk.keys()),
    'MAE': [r['mae'] for r in results_risk.values()],
    'RMSE': [r['rmse'] for r in results_risk.values()],
    'R²': [r['r2'] for r in results_risk.values()]
}).sort_values('R²', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON - AREA RISK SCORE PREDICTION")
print("="*80)
print(comparison_risk.to_string(index=False))

best_model_risk = comparison_risk.iloc[0]['Model']
print(f"\n🏆 Best Model: {best_model_risk}")
print(f"   R²: {results_risk[best_model_risk]['r2']:.4f}")

## 8. Model Comparison & Evaluation <a id='8'></a>

Summary of all models' performance.

In [ ]:
print("="*80)
print("COMPREHENSIVE MODEL EVALUATION SUMMARY")
print("="*80)

summary = pd.DataFrame([
    {
        'Task': 'Crime Category Classification',
        'Best Model': best_model_name,
        'Metric': 'F1-Score',
        'Score': f"{results_category[best_model_name]['f1_score']:.4f}"
    },
    {
        'Task': 'Crime Severity Prediction',
        'Best Model': best_model_sev,
        'Metric': 'AUC-ROC',
        'Score': f"{results_severity[best_model_sev]['auc']:.4f}"
    },
    {
        'Task': 'Weapon Involvement Prediction',
        'Best Model': best_model_weapon,
        'Metric': 'F1-Score',
        'Score': f"{results_weapon[best_model_weapon]['f1_score']:.4f}"
    },
    {
        'Task': 'Crime Occurrence Prediction',
        'Best Model': best_model_ts,
        'Metric': 'R²',
        'Score': f"{results_ts[best_model_ts]['r2']:.4f}"
    },
    {
        'Task': 'Area Risk Score Prediction',
        'Best Model': best_model_risk,
        'Metric': 'R²',
        'Score': f"{results_risk[best_model_risk]['r2']:.4f}"
    }
])

print("\n")
print(summary.to_string(index=False))
print("\n" + "="*80)

## 9. Feature Importance Analysis <a id='9'></a>

Analyze which features are most important for predictions.

In [ ]:
# Feature importance for Crime Category model (if using tree-based)
if hasattr(results_category[best_model_name]['model'], 'feature_importances_'):
    importance_df = pd.DataFrame({
        'Feature': available_features,
        'Importance': results_category[best_model_name]['model'].feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\n" + "="*80)
    print(f"TOP 15 MOST IMPORTANT FEATURES - {best_model_name}")
    print("="*80)
    print(importance_df.head(15).to_string(index=False))
    
    # Visualize
    fig, ax = plt.subplots(figsize=(12, 8))
    top_features = importance_df.head(15)
    ax.barh(range(len(top_features)), top_features['Importance'].values)
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['Feature'].values)
    ax.set_xlabel('Importance', fontsize=12)
    ax.set_title(f'Top 15 Feature Importances - {best_model_name}', fontsize=14, fontweight='bold')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Visualization saved: feature_importance.png")

## 10. Deployment Ready Models <a id='10'></a>

Save the best models for deployment.

In [ ]:
# Save all best models
models_to_save = {
    'crime_category_classifier': {
        'model': results_category[best_model_name]['model'],
        'scaler': scaler,
        'features': available_features
    },
    'crime_severity_classifier': {
        'model': results_severity[best_model_sev]['model'],
        'scaler': scaler_sev,
        'features': available_features
    },
    'weapon_involvement_classifier': {
        'model': results_weapon[best_model_weapon]['model'],
        'scaler': scaler_weap,
        'features': weapon_features
    },
    'crime_occurrence_regressor': {
        'model': results_ts[best_model_ts]['model'],
        'features': ts_features
    },
    'area_risk_regressor': {
        'model': results_risk[best_model_risk]['model'],
        'features': risk_features
    }
}

# Save models
print("\nSaving models...\n")
for name, model_info in models_to_save.items():
    filename = f'{name}_model.pkl'
    joblib.dump(model_info, filename)
    print(f"✓ Saved: {filename}")

# Save label encoders
joblib.dump(label_encoders, 'label_encoders.pkl')
print("✓ Saved: label_encoders.pkl")

print("\n" + "="*80)
print("ALL MODELS SAVED SUCCESSFULLY!")
print("="*80)

## Conclusion

We have successfully developed and evaluated multiple machine learning models for crime prediction:

1. **Crime Category Classification**: Predicts whether a crime is violent, property, or other
2. **Crime Severity Prediction**: Classifies crimes as serious (Part 1) or less serious (Part 2)
3. **Weapon Involvement Prediction**: Determines if a weapon will be involved
4. **Crime Occurrence Prediction**: Forecasts the number of crimes in future time periods
5. **Area Risk Score Regression**: Estimates risk scores for different areas

### Key Findings:

- Tree-based ensemble models (Random Forest, Gradient Boosting) generally performed best
- Feature importance analysis reveals critical factors in crime prediction
- Models achieve strong predictive performance across all tasks
- All models are saved and ready for deployment

### Next Steps:

1. **Deploy models** in production environment
2. **Monitor performance** and retrain with new data
3. **Create API endpoints** for real-time predictions
4. **Integrate with Streamlit dashboard** for interactive predictions
5. **Develop automated retraining pipeline**

---

**All models and visualizations have been saved for deployment and reporting.**